# Efficient Routing

You knew the traveling salesman problem is NP-complete. From wikipedia -- https://en.wikipedia.org/wiki/Travelling_salesman_problem

>The travelling salesman problem (TSP) asks the following question: "Given a list of cities and the distances between each pair of cities, what is the shortest possible route that visits each city exactly once and returns to the origin city?" It is an NP-hard problem in combinatorial optimization, important in operations research and theoretical computer science.

Yet consider a simple everyday example of garbage collection, USPS delivery, meals-on-wheels, or everyday GPS navigation. There is a need for a scalable solution to minimize the distance and maximize resource utlilization. 

When theoretical solutions may be computationally costly, heuristic solutions offer a faster (perhaps semi-optimal) solutions. One such routing heuristic is http://connection.ebscohost.com/c/articles/6691618/minimal-technology-routing-system-meals-wheels by Dr. Bartholdi that uses Hilbert space filling curves to offer short routes to TSP problems.

# Top 20 Cities

Let us suppose the gopher wants to travel to the top 20 US populated cities from New York.

# Data

The top 20 cities data is populated at https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population

In [1]:
# Read data from the web; the top 20 cities
cities = pd.read_html('https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population', header=0)[3].head(20)

# Display a preview of the cities
display(cities)

,2015 rank,City,State[5],2015 estimate,2010 Census,Change,2014 land area,2010 population density,Location
0,1,New York[6],New York,8550405,8175133,7000459040849855290♠+4.59%,7002302600000000000♠302.6 sq mi 783.8 km2,"7004270120000000000♠27,012 per sq mi 10,430 km−2",40°39′51″N 73°56′19″W﻿ / ﻿40.6643°N 73.9385°W﻿...
1,2,Los Angeles,California,3971883,3792621,7000472659936228800♠+4.73%,"7002468700000000000♠468.7 sq mi 1,213.9 km2","7003809200000000000♠8,092 per sq mi 3,124 km−2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...
2,3,Chicago,Illinois,2720546,2695598,6999925508922324480♠+0.93%,7002227600000000000♠227.6 sq mi 589.6 km2,"7004118420000000000♠11,842 per sq mi 4,572 km−2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...
3,4,Houston[7],Texas,2296224,2100263,7000933030768051430♠+9.33%,"7002599600000000000♠599.6 sq mi 1,552.9 km2","7003350100000000000♠3,501 per sq mi 1,352 km−2",29°46′50″N 95°23′11″W﻿ / ﻿29.7805°N 95.3863°W﻿...
4,5,Philadelphia[8],Pennsylvania,1567442,1526006,7000271532353083800♠+2.72%,7002134100000000000♠134.1 sq mi 347.3 km2,"7004113790000000000♠11,379 per sq mi 4,394 km−2",40°00′34″N 75°08′00″W﻿ / ﻿40.0094°N 75.1333°W﻿...
...,...,...,...,...,...,...,...,...,...
15,16,Fort Worth,Texas,833319,741206,7001124274493190830♠+12.43%,7002339800000000000♠339.8 sq mi 880.1 km2,"7003218100000000000♠2,181 per sq mi 842 km−2",32°46′46″N 97°20′47″W﻿ / ﻿32.7795°N 97.3463°W﻿...
16,17,Charlotte,North Carolina,827097,731424,7001130803747210920♠+13.08%,7002297700000000000♠297.7 sq mi 771.0 km2,"7003245700000000000♠2,457 per sq mi 949 km−2",35°12′31″N 80°49′51″W﻿ / ﻿35.2087°N 80.8307°W﻿...
17,18,Seattle,Washington,684451,608660,7001124521079091780♠+12.45%,7001839000000000000♠83.9 sq mi 217.4 km2,"7003725100000000000♠7,251 per sq mi 2,800 km−2",47°37′14″N 122°21′03″W﻿ / ﻿47.6205°N 122.3509°...
18,19,Denver[12],Colorado,682545,600158,7001137275517447070♠+13.73%,7002153000000000000♠153.0 sq mi 396.3 km2,"7003392300000000000♠3,923 per sq mi 1,515 km−2",39°45′42″N 104°52′50″W﻿ / ﻿39.7618°N 104.8806°...


In [2]:
# Compute lat-lng codes and citynames. We are not interested in others
interested_cities = cities[['City', 'Location']].copy()

# Strip any footnote references from the city name
interested_cities['City'] = interested_cities['City'].apply(lambda x: x.split('[')[0])

# Write logic to parse lat lon code
from LatLon import string2latlon
import re
def parse_lat_lon(code_str):
    # Strip the degree symbol
    (lat,lng) = map(lambda x: x.replace('|', ' '), re.sub(r'([NEWS])', r'|\1', code_str.encode('ascii', errors='ignore')).split())
    # Return lat lon parsed from direction
    return string2latlon(lat, lng, 'D% %H')

# Collect only lat long codes
interested_cities['Location'] = interested_cities['Location'].\
    apply(lambda x: parse_lat_lon(x.split('/')[1]))

# Display preview of lat-lng and city
display(interested_cities)

,City,Location
0,New York,"40.6643, -73.9385"
1,Los Angeles,"34.0194, -118.4108"
2,Chicago,"41.8376, -87.6818"
3,Houston,"29.7805, -95.3863"
4,Philadelphia,"40.0094, -75.1333"
...,...,...
15,Fort Worth,"32.7795, -97.3463"
16,Charlotte,"35.2087, -80.8307"
17,Seattle,"47.6205, -122.3509"
18,Denver,"39.7618, -104.8806"


# Encode

Encode the geocode into a "space filling curve index". Google's S2 Sphere is one such code. See http://blog.christianperone.com/2015/08/googles-s2-geometry-on-the-sphere-cells-and-hilbert-curve/ for a detailed explanation

In [3]:
# Encode s2 cell ids for each cities
import s2sphere
from s2sphere import Angle, CellId, LatLng

# Cell ID is a number that resolves a geographic location to a unit sq-cm on the earth with only one long number
get_cellid = lambda latlng: CellId.from_lat_lng(LatLng.from_degrees(latlng[0], latlng[1])).id() if all(latlng) else -1

# Put the CellID in the dataframe
interested_cities['CellID'] = interested_cities['Location'].\
    apply(lambda x: get_cellid(map(float, str(x).split(','))))
    
# Display preview
display(interested_cities)

,City,Location,CellID
0,New York,"40.6643, -73.9385",9926597124620835371
1,Los Angeles,"34.0194, -118.4108",9278182926850512769
2,Chicago,"41.8376, -87.6818",9803823555621206013
3,Houston,"29.7805, -95.3863",9673935061331567749
4,Philadelphia,"40.0094, -75.1333",9927824642483537863
...,...,...,...
15,Fort Worth,"32.7795, -97.3463",9677803118885726507
16,Charlotte,"35.2087, -80.8307",9824215502803758335
17,Seattle,"47.6205, -122.3509",6093393685188758747
18,Denver,"39.7618, -104.8806",9758310259422007871


# Order

Order the travel from New York. Since we are beginning our travel from New York, let us identify next target.

In [4]:
# Compute routine to identify the next closest location from place
# Let us preplan the route
def get_order(cell_list):
    # Push the first cell as the initial state
    visited = [cell_list[0]]
    
    # For each state, aka city, find closest city that has not been visited
    def find_nearest(ids, value):
        # Find closest city to current city where it has not already been visited
        n = [(abs(i-value), i) for i in ids if i != value and i not in visited]
        value = sorted(n)[0][1] if len(n) > 0 else None
        visited.append(value)
        return value
    
    # Find until all cities have been visited
    while True:
        cellId = visited[-1]
        if cellId:
            find_nearest(cell_list, cellId)
        else:
            break
    # Append source city because we need a full circuit
    return pd.DataFrame(list(enumerate([id for id in visited + [visited[0]] if id])), columns=['Order', 'VisitCellID'])

# Compute plan
plan = get_order(interested_cities.CellID.values)

# Display the plan
display(plan)

# Append preferred destinations to the dataframe
routed_cities = interested_cities.merge(plan, left_on='CellID', right_on='VisitCellID').sort_values('Order')

# Display preview
display(routed_cities)

,Order,VisitCellID
0,0,9926597124620835371
1,1,9927824642483537863
2,2,9864492025068926333
3,3,9830039593054841113
4,4,9824215502803758335
...,...,...
16,16,9278182926850512769
17,17,9263396989388013471
18,18,9260949627235681171
19,19,6093393685188758747


,City,Location,CellID,Order,VisitCellID
0,New York,"40.6643, -73.9385",9926597124620835371,0,9926597124620835371
5,Philadelphia,"40.0094, -75.1333",9927824642483537863,1,9927824642483537863
12,Jacksonville,"30.337, -81.6613",9864492025068926333,2,9864492025068926333
14,Indianapolis,"39.7767, -86.1459",9830039593054841113,3,9830039593054841113
17,Charlotte,"35.2087, -80.8307",9824215502803758335,4,9824215502803758335
...,...,...,...,...,...
2,Los Angeles,"34.0194, -118.4108",9278182926850512769,16,9278182926850512769
10,San Jose,"37.2969, -121.8193",9263396989388013471,17,9263396989388013471
13,San Francisco,"37.7751, -122.4193",9260949627235681171,18,9260949627235681171
18,Seattle,"47.6205, -122.3509",6093393685188758747,19,6093393685188758747


# Plotting Route

In [5]:
# Plot the map
import folium
folium.initialize_notebook()

# Find coordinates per city in order
city_coordinates = routed_cities.apply(lambda x: (x.City, list(map(float, str(x.Location).split(','))), x.Order + 1), axis=1).values

# Initialize a map
geomap = folium.Map(location=[32.900908, -97.040335], zoom_start=4) # US map

# Create lines
lines = folium.PolyLine(locations=map(lambda x: x[1], city_coordinates), color='blue', weight=5, opacity=0.3)

# Create markers
for city in city_coordinates:
    marker = folium.CircleMarker(
        location = city[1],
        radius=100,
        popup = "{0} visited {1}".format(city[0], city[2])).add_to(geomap)

# Plot map
geomap.add_child(lines)

# Render
display(geomap)

# Optimized?

Perhaps, does not seem overly circuitous, does it? 
<hr/>